In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset

데이터 : 스마트폰, 이마, 색소침착, 전체데이터<br>
내  용 : 모든라벨 (0 - 5) 6개에 대한 파이토치를 이용한 기본 CNN모델 테스트<br>
결  과 : 한번호  1등급으로 모두 찍는 경향 확인<br>
보  완 : 04. Feasibility of Learning 파일 확인 

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cpu


In [3]:
# 데이터 준비 1. 핸드폰, 전면, 이마, 메타데이터 전체

import pickle

path = "Data/Train_Data_Sets_02.pkl"
with open(path, "rb") as pickle_file:
    train_data_sets = pickle.load(pickle_file)
    print("데이터 로드 완료!!")

데이터 로드 완료!!


In [ ]:
keys = []
for idx, file_name in enumerate(train_data_sets["Metadata"]): 
    pig = train_data_sets["Metadata"][file_name]['annotations']['forehead_pigmentation']
    # 올바른 논리 연산
    if pig == 2:
        keys.append(file_name)
print(keys)

In [4]:
path = "Data/Val_Data_Sets_02.pkl"
with open(path, "rb") as pickle_file:
    test_data_sets = pickle.load(pickle_file)
    print("데이터 로드 완료!!")

데이터 로드 완료!!


In [5]:
# #데이터 구성 확인
# print(train_data_sets.keys())
# print(train_data_sets['Images'].keys())
# print(train_data_sets['Metadata'].keys())
# print(train_data_sets['Images']['0002_03_F_01'])
# print(train_data_sets['Metadata']['0002_03_F_01'])

In [6]:
# 데이터 준비 1.
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((224, 224))])

In [7]:
class CustomDataset(Dataset):
    def __init__(self, data_sets):
        """
        Args:
            data_sets (dict): {'Images': dict, 'Metadata': dict}
        """
        self.images = data_sets['Images']
        self.metadata = data_sets['Metadata']
        self.keys = list(self.images.keys())  # 공통 키 목록
        self.transform = transform

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        key = self.keys[idx]
        image = self.images[key]
        if self.transform:
            image = self.transform(image)
    
        target = torch.tensor(self.metadata[key]['annotations']['forehead_pigmentation'], dtype=torch.float32)  # 타겟 텐서
        return image, target


In [8]:
#데이터셋 생성
dataset = CustomDataset(train_data_sets)
val_dataset = CustomDataset(test_data_sets)

In [9]:
len(dataset)

858

In [10]:
# 첫 5개의 샘플 확인
for idx, (image, target) in enumerate(dataset):
    if idx < 5:
        print(f"샘플 {idx}:")
        print("이미지 텐서 크기:", image.shape)
        print("타겟 값:", target)

샘플 0:
이미지 텐서 크기: torch.Size([3, 224, 224])
타겟 값: tensor(1.)
샘플 1:
이미지 텐서 크기: torch.Size([3, 224, 224])
타겟 값: tensor(1.)
샘플 2:
이미지 텐서 크기: torch.Size([3, 224, 224])
타겟 값: tensor(1.)
샘플 3:
이미지 텐서 크기: torch.Size([3, 224, 224])
타겟 값: tensor(1.)
샘플 4:
이미지 텐서 크기: torch.Size([3, 224, 224])
타겟 값: tensor(0.)


In [11]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)

test_loader = torch.utils.data.DataLoader(val_dataset, batch_size=4, shuffle=True)

In [12]:
for X, y in train_loader:
    print(X.shape, y.shape)
    break

torch.Size([4, 3, 224, 224]) torch.Size([4])


In [13]:
56*56*192

602112

In [14]:
class SkinNet(nn.Module):
    def __init__(self):
        super(SkinNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=24, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=24, out_channels=192, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(56*56*192, 1024)
        # self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 6)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)

        x = x.view(-1, 56*56*192)
        x = self.fc1(x)
        x = F.relu(x)
        # x = self.fc2(x)
        # x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        x = F.relu(x)
        x = self.fc5(x)
        x = F.log_softmax(x, dim = 1)
        return x


In [15]:
model = SkinNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

print(model)

SkinNet(
  (conv1): Conv2d(3, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(24, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=602112, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=6, bias=True)
)


In [16]:
def train(model, train_loader, optimizer):
    loss_total = 0
    correct_total = 0
    model.train()
    for idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device).long()

        optimizer.zero_grad()

        output = model(images)
        loss = criterion(output, labels)

        loss.backward()
        optimizer.step()

        # 누적 손실 및 정확도 계산
        loss_total += loss.item()
        predicted_values = output.max(1, keepdim=True)[1]
        correct = predicted_values.eq(labels.view_as(predicted_values)).sum().item()
        correct_total += correct

        if idx % 10 == 0:
            print(f"Batch : {idx}, Loss : {loss.item()}")

        # 에폭 종료 후 평균 손실 및 정확도 계산
    loss_total /= len(train_loader)  # 배치 개수로 나누어 평균 계산
    accuracy = correct_total / len(train_loader.dataset)  # 전체 데이터셋에서 정확도 계산
    
    # 마지막 배치 후 결과 출력
    print(f"Epoch Finished - Loss: {loss_total:.4f}, Accuracy: {accuracy:.4f}")
    
    return loss_total, accuracy

In [17]:
def evaluate(model, test_loader):
    loss_total = 0
    correct_total = 0
    model.eval() # evaluation mode로 설정 -> batch-normalization, drop-out 수행 중지
    with torch.no_grad(): # 가중치 업데이트 수행 중지
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device).long()
            output = model(images)
            loss = criterion(output, labels).item()
            loss_total += loss
            predicted_values = output.max(1, keepdim=True)[1]
            correct = predicted_values.eq(labels.view_as(predicted_values)).sum().item()
            correct_total += correct

    loss_total /= ( len(val_dataset) / 4 )
    accuracy = correct_total / len(val_dataset)

    return loss_total, accuracy



In [ ]:
# 학습 실행

for epoch in range(10):    
    train(model, train_loader, optimizer)
    val_loss, val_accuracy = evaluate(model, test_loader)
    print(f"Epoch : {epoch + 1}, val_Loss: {val_loss}, val_Accuracy : {val_accuracy}")